In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a """
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer 1__

Reinforcement Learning is most successful when we find a good compromise between **exploring** the environment to discover successful strategies and **exploiting** what we already know about the environment. 

So $\epsilon$ is essential as it sets the probability with which the agent takes a random action, hence "exploring" a new possible strategy. With some luck this might improve the reward that can be accumulated. Tuning the value of $\epsilon$ will allow us to get the best compromise between exploring and exploiting.

This is why $\epsilon$ is only used when training the agent. In the test phase, we want to be purely exploiting previous knowledge which we hope to be sufficient.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        self.grid_size = grid_size + 4  # size of the game board, including edges
        self.max_time = max_time  # maximum time alloted to play the game
        self.temperature = temperature  # probability a square contains cheese or poison 

        # board : array of rewards left to collect
        # position : keeps track of the rat's position in the board
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # set the timer to 0
        self.t = 0

        self.scale = 16  # number of pixels per square of the grid
        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        # Colors used
        land_col = (255,248,220)
        cheese_col = (255, 215, 0)
        poison_col = (50,205,50)
        rat_col = (47, 79, 79)
        boundary_col = (25, 25, 112)
        # make RGB representation of the game board
        b = np.tile(land_col, (self.grid_size, self.grid_size, 1))  # default is grey
        b[self.board > 0,:] = cheese_col  # cheese is golden
        b[self.board < 0,:] = poison_col  # poison is mucky green
        b[self.x,self.y,:] = rat_col  # current position in white
        # edges are black
        b[-2:,:,:] = boundary_col
        b[:,-2:,:] = boundary_col
        b[:2,:,:] = boundary_col
        b[:,:2,:] = boundary_col
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        # game board position grid: 0 by default or -1 if it's an edge
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        
        # instantiate reward
        reward = 0
        drowning_penalty = 0

        # position the rat is leaving set to 1 on game board
        self.position[self.x, self.y] = 1
        if action == 0:  # go right
            if self.x == self.grid_size - 3:  # but if at edge go left
                self.x = self.x - 1; reward += drowning_penalty
            else:
                self.x = self.x + 1
        elif action == 1:  # go left
            if self.x == 2:  # but if at edge go right
                self.x = self.x + 1; reward += drowning_penalty
            else:
                self.x = self.x - 1
        elif action == 2:  # go down
            if self.y == self.grid_size - 3:  # but if at edge go up
                self.y = self.y - 1; reward += drowning_penalty
            else:
                self.y = self.y + 1
        elif action == 3:  # go up
            if self.y == 2:  # but if at edge go down
                self.y = self.y + 1; reward += drowning_penalty
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        # increment time and go to game over if time is up
        self.t += 1
        game_over = self.t > self.max_time
        
        # gather reward (cheese or poison)
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        
        # 'state' encapsulates both the reward grid and position grid
        state = np.dstack((self.board, self.position))
        # only keep state information the rat can see (i.e. up to 2 squares away)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        # parachute the rat to a random spot on the island, PUBG style
        self.x = np.random.randint(3, self.grid_size - 2)
        self.y = np.random.randint(3, self.grid_size - 2)

        # set spots with cheese (bonus +.5) and poison (malus -1)
        # both happen with probability 'temperature'
        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)
        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)
        # make sure a spot doesn't contain both (cheese supercedes poison)
        malus[bonus > 0] = 0
        # store the reward markers
        self.board = bonus + malus
        # ensure there is no reward on the start position
        self.board[self.x, self.y] = 0
        
        # instantiate the variable we draw
        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))

        # game board position grid: 0 by default or -1 if it's an edge
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:] = -1
        self.position[:,0:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.t = 0

        # 'state' encapsulates both the reward grid and position grid
        state = np.dstack((self.board, self.position))
        # only keep state information the rat can see (i.e. up to 2 squares away)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 15  # size of the game board
T = 200  # maximum time alloted to play the game
temperature = 0.3  # probability a square contains cheese or poison
epochs_train = 2 # set small when debugging
epochs_test = 2 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer 2__ 

```position``` records the position of the rat. It is a 2D grid of 0s except for the spots corresponding to places beyond the game board boundaries (which are set to $-1$) and the current position of the rat (set to $1$). The boundaries are to account for the fact that the rat can see two squares away. Later on we will keep track of the squares previously visited in this array, but not for the moment.

```board``` keeps track of the rewards awaiting the rat. It is a 2D grid of 0s except for squares containing bonuses (cheese) which are set to $0.5$ and those containing maluses (poison) which are set to $-1$. Both bonuses and maluses appear with a probability equal to the value of ```temperature```, and a spot on the board cannot contain both cheese and poison. When the rat passes on a spot, the reward is collected and set to 0.

Note that we edited the code in several places: 
* a bug was fixed when setting the boundaries in ```position``` to $-1$. (```self.position[:,-2:] = -1``` instead of ```self.position[-2:,:] = -1```, which was already stated)
* another minor bug was fixed in the ```reset()``` method when instantiating the position of the rat: the upper boundary of ```np.random.randint()``` was incremented as it is exclusive.
* the long ```np.concatenate()``` statement in the definition of ```state``` was replaced by a more concise call to ```np.dstack()```
* the colors of the grid for the videos were changed to be a bit more evocative
* many comments were added + a few small style changes
* a penalty for trying to go off the island is added (we later notice that the rat often oscillates at the island border, so this is to avoid this)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs + 1):
        
        # we restart to a fresh game
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action and gets the results
            action = agent.act(state)  
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0: win += reward
            if reward < 0: lose -= reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score: {}"
              .format(win, lose, score / (1 + e)))
    print('Final score: ' + str(score / epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 9.5/14.0. Average score: -2.25
Win/lose count 8.0/14.0. Average score: -3.5
Final score: -5.25


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer 5__ 

__Part 1__: From the definition further up of the Action-Value function $Q^\pi$ of policy $\pi$:

$$ Q^\pi(s,a)=E_{p^{\pi}}\left[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] $$

Note that we can increment the $t$ and correspondingly remove a $\gamma$ factor without changing things:

$$ Q^\pi(s',a')=\frac{1}{\gamma} E_{p^{\pi}}\left[\sum_{1\leq t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'\right] $$

We now take the first equation again and separate the first term from the summation and use linearity of expectation:

$$ Q^\pi(s,a)=E_{p^{\pi}}[r(s, a)] + E_{p^{\pi}}\left[\sum_{1\leq t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] $$

Then we use the Markov property to remove $a_0$ and $s_0$ from the equation:

$$ Q^\pi(s,a)=E_{p^{\pi}}[r(s, a)] + E_{(s',a')\sim p(.|s,a)}  \left[\sum_{1\leq t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'\right]$$

Which enables to use the second equation and find our recursive expression for the Action-Value function:

$$ Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}\left[r(s,a)+\gamma Q^{\pi}(s',a')\right] $$

__Part 2__: $Q^{*}(s,a)$ refers to our expected return if:
* we first take action $a$ 
* we subsequently choose only optimal actions defined as $a^* = \text{argmax}_{a'} Q^{*}(s',a')$

Similarly $\max_{a'} Q^{*}(s',a')$ refers to our expected return if we start straight away with an optimal action and continue choosing optimal actions.

So finding $Q^{*}(s,a)$ is the same as:
* finding our first expected reward $E[r(s,a)]$
* receiving the next state $s'\sim \pi^*(.|s,a)$ 
* given $s'$ finding the expectation if we only take optimal actions—i.e. adding $\max_{a'} Q^{*}(s',a')$
* adding the expectation from the line above to our result once multiplied by the time discount factor $\gamma$ 

In one formula, we get the desired result:

$$ Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')\right] $$

__Part 3__: Ideally our loss would be:

$$ \mathcal{L}(\theta) = \Vert Q^*(s,a) - Q(s, a, \theta)\Vert^2$$

But that would of course assume we already know $Q^*(s,a)$. But if we suppose the above for an instant and use our previous result, we get:

$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r + \gamma\max_{a'}Q^*(s',a')-Q(s,a,\theta)\Vert^{2}$$

Hence the closer we get to $Q(s,a,\theta) = Q^*(s,a)$, the closer we get to our actual loss function below being zero:
$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$

Note that no other function than $Q^*(s,a)$ can bring the above to 0 since there is only one fixed point to the _Optimal Bellman Equation_. Hence this loss function is quite plausible.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
from collections import deque
from random import sample

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen=max_memory)

    def remember(self, m):
        # the first element is automagically removed if needed
        self.memory.append(m)

    def random_access(self, n=1):
        # sample n slots of memory at once (n bounded by memory size)
        n = min(n, len(self.memory))
        return sample(self.memory, n)

***
The pipeline we will use for training is given below:

In [133]:
def train(agent, env, epoch, prefix='', explore=False, epsilon_factor=.9):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Decrease epsilon exponentially
            if explore: agent.set_epsilon(agent.epsilon * epsilon_factor)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            
            # Account for the explore mode that is used later
            if explore:
                state, reward, game_over = env.act(action, train=True)
            else:
                state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4 every 10 epochs
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.1f})"
              .format(e, epoch, loss, win, lose, win - lose))
        agent.save(name_weights=prefix + 'model.h5', name_model=prefix + 'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [18]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=.1, memory_size=100, batch_size=16, n_state=2, target_clip=3):
        super(DQN, self).__init__(epsilon=epsilon)

        self.discount = 0.99  # Discount for Q learning
        self.target_clip = target_clip  # bound for expected reward values
        self.grid_size = grid_size
        
        # number of grids the environment gives us
        # Here the 'position' and 'board' grid
        self.n_state = n_state

        # Size of total memory and batches extracted
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

        
    def learned_act(self, s):
        predicted_rewards = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        return np.argmax(predicted_rewards)

    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # first memorize the states
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # take a random batch from memory and format the data
        batch = self.memory.random_access(self.batch_size)
        
        # instantiate containers for memory batch data
        input_states = np.zeros((len(batch), 5, 5, self.n_state))
        next_states = np.zeros((len(batch), 5, 5, self.n_state))
        actions = np.zeros(len(batch)).astype('int')
        rewards = np.zeros(len(batch))
        game_overs = np.zeros(len(batch))
        target_q = np.zeros((len(batch), 4))
        
        # unpack batch data
        for i, datum in enumerate(batch):
            input_states[i], next_states[i], actions[i], rewards[i], game_overs[i] = datum
        
        # store what we know about future rewards
        target_q = self.model.predict(input_states)
        next_q = self.model.predict(next_states)
        
        for i, (action, reward, game_over, q) in enumerate(zip(actions, rewards, game_overs, next_q)):
            #print(i, action, reward, game_over, q)
            target_q[i, action] = reward + (not game_over) * self.discount * np.max(q)

        # set bounds for expected reward values to avoid exploding gradients
        target_q = np.clip(target_q, -self.target_clip, self.target_clip)

        # batch train and return loss
        return self.model.train_on_batch(input_states, target_q)

    
    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # Fully Connected NN Model
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(64),
            BatchNormalization(), Activation('relu'),
            Dense(64),
            BatchNormalization(), Activation('relu'),
            #Dense(),
            #BatchNormalization(), Activation('relu'),
            Dense(self.n_action), 
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        #model.summary()
        

In [19]:
epochs_train = 200
T = 200

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, target_clip=10)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train{}.mp4'.format((epochs_train // 10) * 10)))

Epoch 001/200 | Loss 0.0110 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/200 | Loss 1.4275 | Win/lose count 5.0/10.0 (-5.0)
Epoch 003/200 | Loss 8.3439 | Win/lose count 5.0/3.0 (2.0)
Epoch 004/200 | Loss 2.9545 | Win/lose count 1.5/1.0 (0.5)
Epoch 005/200 | Loss 3.4141 | Win/lose count 5.0/2.0 (3.0)
Epoch 006/200 | Loss 0.1952 | Win/lose count 8.5/5.0 (3.5)
Epoch 007/200 | Loss 0.0589 | Win/lose count 5.5/9.0 (-3.5)
Epoch 008/200 | Loss 0.0741 | Win/lose count 7.5/8.0 (-0.5)
Epoch 009/200 | Loss 0.0540 | Win/lose count 8.5/6.0 (2.5)
Epoch 010/200 | Loss 0.0330 | Win/lose count 7.5/6.0 (1.5)
Epoch 011/200 | Loss 0.0237 | Win/lose count 8.0/12.0 (-4.0)
Epoch 012/200 | Loss 0.0206 | Win/lose count 3.5/8.0 (-4.5)
Epoch 013/200 | Loss 0.0346 | Win/lose count 9.0/5.0 (4.0)
Epoch 014/200 | Loss 0.1170 | Win/lose count 7.5/8.0 (-0.5)
Epoch 015/200 | Loss 0.0335 | Win/lose count 5.0/4.0 (1.0)
Epoch 016/200 | Loss 0.2126 | Win/lose count 9.0/10.0 (-1.0)
Epoch 017/200 | Loss 0.1156 | Win/lose count 6

Epoch 139/200 | Loss 0.0142 | Win/lose count 7.0/4.0 (3.0)
Epoch 140/200 | Loss 0.0120 | Win/lose count 9.0/7.0 (2.0)
Epoch 141/200 | Loss 0.0156 | Win/lose count 7.5/3.0 (4.5)
Epoch 142/200 | Loss 0.0186 | Win/lose count 13.0/5.0 (8.0)
Epoch 143/200 | Loss 0.0129 | Win/lose count 6.5/3.0 (3.5)
Epoch 144/200 | Loss 0.0101 | Win/lose count 9.5/5.0 (4.5)
Epoch 145/200 | Loss 0.0070 | Win/lose count 21.5/3.0 (18.5)
Epoch 146/200 | Loss 0.0063 | Win/lose count 15.5/2.0 (13.5)
Epoch 147/200 | Loss 0.0093 | Win/lose count 10.5/1.0 (9.5)
Epoch 148/200 | Loss 0.0082 | Win/lose count 6.5/2.0 (4.5)
Epoch 149/200 | Loss 0.0113 | Win/lose count 11.0/0 (11.0)
Epoch 150/200 | Loss 0.1589 | Win/lose count 12.0/5.0 (7.0)
Epoch 151/200 | Loss 0.0243 | Win/lose count 6.5/4.0 (2.5)
Epoch 152/200 | Loss 0.0165 | Win/lose count 12.5/2.0 (10.5)
Epoch 153/200 | Loss 0.1569 | Win/lose count 13.5/4.0 (9.5)
Epoch 154/200 | Loss 0.0213 | Win/lose count 10.5/3.0 (7.5)
Epoch 155/200 | Loss 0.0134 | Win/lose count 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [26]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=.1, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)
        
        # CNN model
        model = Sequential([
            Conv2D(16, (2, 2), padding='valid', input_shape=(5, 5, self.n_state)),
            BatchNormalization(), Activation('relu'),
            Conv2D(16, (2, 2), padding='valid'),
            BatchNormalization(), Activation('relu'),
            Flatten(),
            Dense(self.n_action), 
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        #model.summary()


In [27]:
epochs_train = 200
T = 200

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, target_clip=10)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 001/200 | Loss 0.0119 | Win/lose count 8.0/11.0 (-3.0)
Epoch 002/200 | Loss 0.0163 | Win/lose count 7.0/4.0 (3.0)
Epoch 003/200 | Loss 0.0372 | Win/lose count 8.5/7.0 (1.5)
Epoch 004/200 | Loss 0.0225 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/200 | Loss 0.6319 | Win/lose count 9.5/7.0 (2.5)
Epoch 006/200 | Loss 0.0580 | Win/lose count 7.5/5.0 (2.5)
Epoch 007/200 | Loss 0.0180 | Win/lose count 10.0/14.0 (-4.0)
Epoch 008/200 | Loss 0.0197 | Win/lose count 4.0/10.0 (-6.0)
Epoch 009/200 | Loss 0.0128 | Win/lose count 11.5/6.0 (5.5)
Epoch 010/200 | Loss 0.0393 | Win/lose count 7.0/7.0 (0.0)
Epoch 011/200 | Loss 0.0285 | Win/lose count 7.5/2.0 (5.5)
Epoch 012/200 | Loss 0.0155 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/200 | Loss 0.0378 | Win/lose count 6.0/4.0 (2.0)
Epoch 014/200 | Loss 0.1000 | Win/lose count 3.5/7.0 (-3.5)
Epoch 015/200 | Loss 0.0443 | Win/lose count 3.5/8.0 (-4.5)
Epoch 016/200 | Loss 0.0271 | Win/lose count 7.5/5.0 (2.5)
Epoch 017/200 | Loss 0.0514 | Win/lose count 3

Epoch 137/200 | Loss 0.0227 | Win/lose count 23.5/12.0 (11.5)
Epoch 138/200 | Loss 0.0414 | Win/lose count 22.5/13.0 (9.5)
Epoch 139/200 | Loss 0.0226 | Win/lose count 15.0/6.0 (9.0)
Epoch 140/200 | Loss 0.0320 | Win/lose count 25.0/13.0 (12.0)
Epoch 141/200 | Loss 0.0184 | Win/lose count 13.5/11.0 (2.5)
Epoch 142/200 | Loss 0.0264 | Win/lose count 29.0/8.0 (21.0)
Epoch 143/200 | Loss 0.0325 | Win/lose count 17.0/9.0 (8.0)
Epoch 144/200 | Loss 0.0151 | Win/lose count 15.0/7.0 (8.0)
Epoch 145/200 | Loss 0.0358 | Win/lose count 19.5/11.0 (8.5)
Epoch 146/200 | Loss 0.2314 | Win/lose count 21.5/6.0 (15.5)
Epoch 147/200 | Loss 0.0352 | Win/lose count 23.0/6.0 (17.0)
Epoch 148/200 | Loss 0.0512 | Win/lose count 23.0/6.0 (17.0)
Epoch 149/200 | Loss 0.0249 | Win/lose count 20.5/8.0 (12.5)
Epoch 150/200 | Loss 0.0209 | Win/lose count 13.5/8.0 (5.5)
Epoch 151/200 | Loss 0.1468 | Win/lose count 22.0/9.0 (13.0)
Epoch 152/200 | Loss 0.0316 | Win/lose count 20.5/8.0 (12.5)
Epoch 153/200 | Loss 0.252

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [28]:
epochs_test = 10

env = Environment(size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon=.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5', name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(grid_size=size, lr=.1, epsilon=.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/3.0. Average score: 1.0
Win/lose count 4.0/2.0. Average score: 1.3333333333333333
Win/lose count 9.0/2.0. Average score: 2.75
Win/lose count 17.0/0. Average score: 5.6
Win/lose count 7.5/3.0. Average score: 5.416666666666667
Win/lose count 11.0/1.0. Average score: 6.071428571428571
Win/lose count 14.0/4.0. Average score: 6.5625
Win/lose count 18.0/2.0. Average score: 7.611111111111111
Win/lose count 8.0/3.0. Average score: 7.35
Win/lose count 21.5/4.0. Average score: 8.272727272727273
Final score: 9.1
Test of the FC
Win/lose count 6.5/6.0. Average score: 0.25
Win/lose count 5.5/8.0. Average score: -0.6666666666666666
Win/lose count 2.0/4.0. Average score: -1.0
Win/lose count 6.5/5.0. Average score: -0.5
Win/lose count 4.0/14.0. Average score: -2.0833333333333335
Win/lose count 5.0/10.0. Average score: -2.5
Win/lose count 3.0/5.0. Average score: -2.4375
Win/lose count 3.5/10.0. Average score: -2.888888888888889
Win/lose count 5.5/5.0. Average score: -2

In [29]:
HTML(display_videos('cnn_test10.mp4'))

In [30]:
HTML(display_videos('fc_test10.mp4'))

__Answer 9__

Both agents we have trained fall into a problematic _local minimum_: they eventually start oscillating back and forth or turning in circles, meaning they do not explore enough. Indeed going onto unknown ground contains more risk than gain (poison gives a bigger penalty than cheese gives as a bonus), so the agent decides to stick to places it knows. This can be fixed by implementing the memory of visited places seen in the next part.

When we raise the temperature, the agent is surrounded with cheese and has much less trouble winning ; but when the rewards are sparser, being disinclined to explore, the agent does not go very far.

As for differences between the dense network and the convolutional network, we see in big difference in winning scores that the CNN was more successful in teaching the rat to avoid poison squares and seek out the cheese it sees.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [137]:
def train_explore(agent, env, epoch, prefix='', epsilon_factor=.9):
    train(agent, env, epoch, prefix, True, epsilon_factor)

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super().__init__(grid_size, max_time, temperature)
        
        # where the rat has already been
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
    def reset(self):
        state = super().reset()
        
        # put the history of visited places back to 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        # pass this on with the rest of the state information
        malus_position_window = self.malus_position[self.x - 2:self.x + 3, self.y - 2:self.y + 3]
        state = np.dstack((state, malus_position_window))
        # record that the rat has passed here
        self.malus_position[self.x, self.y] += .1
        
        return state
    
    def act(self, action, train=False):
        state, reward, game_over = super().act(action)
        
        # add a malus if the rat revisits a square
        if train:
            reward -= self.malus_position[self.x, self.y]
        
        # pass this on with the rest of the state information
        malus_position_window = self.malus_position[self.x - 2:self.x + 3, self.y - 2:self.y + 3]
        state = np.dstack((state, malus_position_window))
        # record that the rat has passed here
        self.malus_position[self.x, self.y] += .1
        
        return state, reward, game_over

In [138]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 001/200 | Loss 0.0334 | Win/lose count 11.5/64.0 (-52.5)
Epoch 002/200 | Loss 0.0337 | Win/lose count 14.5/53.4 (-38.9)
Epoch 003/200 | Loss 0.0269 | Win/lose count 10.5/58.0 (-47.5)
Epoch 004/200 | Loss 0.0296 | Win/lose count 15.5/47.8 (-32.3)
Epoch 005/200 | Loss 0.1320 | Win/lose count 11.5/40.9 (-29.4)
Epoch 006/200 | Loss 0.0179 | Win/lose count 19.0/38.0 (-19.0)
Epoch 007/200 | Loss 0.0501 | Win/lose count 21.5/25.9 (-4.4)
Epoch 008/200 | Loss 0.0399 | Win/lose count 23.0/26.7 (-3.7)
Epoch 009/200 | Loss 0.0325 | Win/lose count 22.5/20.4 (2.1)
Epoch 010/200 | Loss 0.0196 | Win/lose count 21.5/22.4 (-0.9)
Epoch 011/200 | Loss 0.0253 | Win/lose count 22.5/22.0 (0.5)
Epoch 012/200 | Loss 0.0243 | Win/lose count 28.0/22.2 (5.8)
Epoch 013/200 | Loss 0.0347 | Win/lose count 19.5/26.5 (-7.0)
Epoch 014/200 | Loss 0.0284 | Win/lose count 23.5/23.3 (0.2)
Epoch 015/200 | Loss 0.0310 | Win/lose count 18.0/28.4 (-10.4)
Epoch 016/200 | Loss 0.0367 | Win/lose count 24.5/15.3 (9.2)
Epoch 

Epoch 134/200 | Loss 0.0467 | Win/lose count 27.5/10.2 (17.3)
Epoch 135/200 | Loss 0.0249 | Win/lose count 32.5/13.0 (19.5)
Epoch 136/200 | Loss 0.0286 | Win/lose count 29.5/10.9 (18.6)
Epoch 137/200 | Loss 0.0277 | Win/lose count 26.5/15.9 (10.6)
Epoch 138/200 | Loss 0.0408 | Win/lose count 32.0/9.4 (22.6)
Epoch 139/200 | Loss 0.0247 | Win/lose count 32.5/17.0 (15.5)
Epoch 140/200 | Loss 0.0357 | Win/lose count 31.5/10.1 (21.4)
Epoch 141/200 | Loss 0.0428 | Win/lose count 27.0/9.7 (17.3)
Epoch 142/200 | Loss 0.0335 | Win/lose count 30.5/11.4 (19.1)
Epoch 143/200 | Loss 0.0211 | Win/lose count 27.5/8.5 (19.0)
Epoch 144/200 | Loss 0.0181 | Win/lose count 26.0/8.0 (18.0)
Epoch 145/200 | Loss 0.0497 | Win/lose count 29.5/12.7 (16.8)
Epoch 146/200 | Loss 0.0199 | Win/lose count 27.0/10.1 (16.9)
Epoch 147/200 | Loss 0.0147 | Win/lose count 29.5/10.1 (19.4)
Epoch 148/200 | Loss 0.0138 | Win/lose count 30.0/10.4 (19.6)
Epoch 149/200 | Loss 0.0179 | Win/lose count 32.0/10.0 (22.0)
Epoch 150/20

In [139]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 27.0/5.0. Average score: 11.0
Win/lose count 36.5/6.0. Average score: 17.5
Win/lose count 31.0/3.0. Average score: 20.125
Win/lose count 34.5/4.0. Average score: 22.2
Win/lose count 33.0/5.0. Average score: 23.166666666666668
Win/lose count 35.5/6.0. Average score: 24.071428571428573
Win/lose count 33.0/6.0. Average score: 24.4375
Win/lose count 30.0/3.0. Average score: 24.72222222222222
Win/lose count 29.5/2.0. Average score: 25.0
Win/lose count 34.5/5.0. Average score: 25.40909090909091
Final score: 27.95


__Answer 10__

At first we still observed the agent ending up with oscillatory behaviour. To improve this, instead of giving the rat a fixed penalty for going on a square it already visited (as suggested), we made the penalty proportional to the number of times the rat has visited the square: turning in circles is now a heavily penalized startegy.

Implementing these extra features to force the rat to explore proved extremely successful. Now in most of the games we see the rat constantly exploring new areas, avoiding almost all the poison squares and eating almost all the cheese.

Note that the success of this method is due to the number of squares available on the map being about the same as the time given to the rat to play. If it were much lower our rat would have to end up choosing poison over seeing the same squares again, so our implementation would have to be modified again.

***